## Install Dependencies  
Install FastAPI, cvlib, image processing libraries, CPU-only TensorFlow, and the multipart parser required by FastAPI.

```bash
pip install fastapi uvicorn cvlib opencv-python numpy tensorflow-cpu python-multipart
```

## Imports and Application Setup
Import required libraries and create the FastAPI application instance.

In [24]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import Response
import cv2
import cvlib as cv
import numpy as np
import uvicorn

app = FastAPI(
    title="YOLO Object Detector with cvlib",
    description="Detect objects using cvlib's YOLOv3-tiny wrapper via FastAPI",
    version="1.0"
)

## Define the /predict Endpoint
Accept an uploaded image, run YOLO detection via cvlib, draw bounding boxes, and return the annotated image.

In [25]:
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    """
    - Read uploaded image bytes
    - Decode to OpenCV BGR image
    - Perform object detection with cvlib.detect_common_objects (yolov3-tiny)
    - Draw bounding boxes and labels
    - Encode annotated image as JPEG and return it
    """
    # Read file bytes into memory
    contents = await file.read()
    nparr = np.frombuffer(contents, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Perform object detection
    bbox, labels, confidences = cv.detect_common_objects(image, model="yolov3-tiny")

    # Annotate image
    output_image = cv.object_detection.draw_bbox(image, bbox, labels, confidences)

    # Encode to JPEG
    _, encoded = cv2.imencode(".jpg", output_image)
    return Response(content=encoded.tobytes(), media_type="image/jpeg")